In [1]:
import os
import sys

current_dir = os.path.dirname(os.path.realpath('__file__'))
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from apikey import apikey, langchain_apikey

os.environ['OPENAI_API_KEY'] = apikey
os.environ['LANGCHAIN_API_KEY'] = langchain_apikey
os.environ["LANGCHAIN_TRACING_V2"] = "true"


# 1.记录Traces

## 1-1.直接通过LangChain

In [2]:
# No extra code is needed to log a trace to LangSmith when using LangChain Python.
# Just run your LangChain code as you normally would with the LANGCHAIN_TRACING_V2 environment variable set to 'true' and the LANGCHAIN_API_KEY environment variable set to your API key.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 使用 LangChain Python 时，无需额外的代码即可记录对 LangSmith 的跟踪。
# 只需像往常一样运行您的 LangChain 代码，将 LANGCHAIN_TRACING_V2 环境变量设置为“true”，并将 LANGCHAIN_API_KEY 环境变量设置为您的 API 密钥。


In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are a helpful assistant. Please respond to the user's request only based on the given context.")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Question: {question}\nContext: {context}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10c2f0590>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10c231810>, openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()

In [4]:
question = "Can you summarize this morning's meetings?"
context = "During this morning's meeting, we solved all world conflict."
chain.invoke({"question": question, "context": context})

"During this morning's meeting, all world conflict was resolved."

## 1-2.使用 Python SDK

In [5]:
# To run the example below, ensure the environment variable OPENAI_API_KEY is set
from typing import Any, Iterable
import openai
from langsmith import traceable
from langsmith.run_trees import RunTree
from langsmith.wrappers import wrap_openai



### OPTION 1: Use RunTree API (more explicit) ###


In [ ]:
# This can be a user input to your app
question = "Can you summarize this morning's meetings?"
# Create a top-level run
pipeline = RunTree(
    name="Chat Pipeline Run Tree",
    run_type="chain",
    inputs={"question": question}
)

# This can be retrieved in a retrieval step
context = "During this morning's meeting, we solved all world conflict."

messages = [
    { "role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context." },
    { "role": "user", "content": f"Question: {question}\nContext: {context}"}
]

# Create a child run
child_llm_run = pipeline.create_child(
    name="OpenAI Call",
    run_type="llm",
    inputs={"messages": messages},
)

# Generate a completion
client = openai.Client()
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages
)

# End the runs and log them
child_llm_run.end(outputs=chat_completion)
child_llm_run.post()

pipeline.end(outputs={"answer": chat_completion.choices[0].message.content})
pipeline.post()

### OPTION 2: Use traceable decorator and OpenAI Client ###

In [6]:
# Optional: wrap the openai client to add tracing directly
# This can also be done with a traceable decorator
client = wrap_openai(openai.Client())

@traceable(run_type="tool", name="Retrieve Context")
def my_tool(question: str) -> str:
    return "During this morning's meeting, we solved all world conflict."




In [7]:
@traceable(name="Chat Pipeline Traceable")
def chat_pipeline(question: str):
    context = my_tool(question)
    messages = [
        { "role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context." },
        { "role": "user", "content": f"Question: {question}\nContext: {context}"}
    ]
    chat_completion = client.chat.completions.create(
        model="gpt-3.5-turbo", messages=messages
    )
    return chat_completion.choices[0].message.content

chat_pipeline("Can you summarize this morning's meetings?")

"During this morning's meeting, world conflicts were resolved."

In [ ]:
# cc：这里的核心是通过 traceable 装饰器，整体来看相比于 RunTree 要更加方便一些；

## 1-3. 使用python API方式

In [8]:
import openai
import requests
from datetime import datetime
from uuid import uuid4

In [11]:
# Send your API Key in the request headers
# headers = {"x-api-key": "<YOUR API KEY>"}
headers = {"x-api-key": langchain_apikey}


In [9]:
def post_run(run_id, name, run_type, inputs, parent_id=None):
    """Function to post a new run to the API."""
    data = {
        "id": run_id.hex,
        "name": name,
        "run_type": run_type,
        "inputs": inputs,
        "start_time": datetime.utcnow().isoformat(),
    }
    if parent_id:
        data["parent_run_id"] = parent_id.hex
    requests.post(
        "https://api.smith.langchain.com/runs",
        json=data,
        headers=headers
    )

In [10]:
def patch_run(run_id, outputs):
    """Function to patch a run with outputs."""
    requests.patch(
        f"https://api.smith.langchain.com/runs/{run_id}",
        json={
            "outputs": outputs,
            "end_time": datetime.utcnow().isoformat(),
        },
        headers=headers,
    )

In [13]:

# This can be a user input to your app
question = "Can you summarize this morning's meetings?"

# This can be retrieved in a retrieval step
context = "During this morning's meeting, we solved all world conflict."
messages = [
    {"role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context."},
    {"role": "user", "content": f"Question: {question}\nContext: {context}"}
]

In [14]:
# Create parent run
parent_run_id = uuid4()
post_run(parent_run_id, "Chat Pipeline", "chain", {"question": question})

# Create child run
child_run_id = uuid4()
post_run(child_run_id, "OpenAI Call", "llm", {"messages": messages}, parent_run_id)

# Generate a completion
client = openai.Client()
chat_completion = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)

# End runs
patch_run(child_run_id, chat_completion.dict())
patch_run(parent_run_id, {"answer": chat_completion.choices[0].message.content})

/var/folders/6r/0511lkxx3c3cmf9jbmg0q27r0000gn/T/ipykernel_65306/33855674.py:14: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  patch_run(child_run_id, chat_completion.dict())


# 2.通过 web链接来看Traces

链接：https://smith.langchain.com/o/1040bdf6-7131-5c03-b7be-fb1b4779027f/projects/p/5da73c3f-abb4-4b8e-8af4-51321f6b206a?timeModel=%7B%22duration%22%3A%227d%22%7D

# 3.设置跟踪的采样率:

要对记录到 LangSmith 的跟踪数进行下采样，请将 LANGCHAIN_TRACING_SAMPLING_RATE 环境变量设置为介于 0（无跟踪）和 1（所有跟踪）之间的任何浮点数。

这需要 python SDK 版本 >= 0.0.84 和 JS SDK 版本 >= 0.0.64。例如，设置以下环境变量将过滤掉 25% 的跟踪：

export LANGCHAIN_TRACING_SAMPLING_RATE=0.75

这适用于可跟踪的装饰器和 RunTree 对象。


# 4.关闭 Tracing

如果您决定不再要跟踪运行，则可以删除配置为首先开始跟踪的环境变量。通过取消设置LANGCHAIN_TRACING_V2环境变量，跟踪将不再记录到 LangSmith。请注意，这目前不会影响 RunTree 对象。

This setting works both with LangChain and the LangSmith SDK, in both Python and TypeScript.



# 5.获取已记录运行的Run ID

In [15]:
import openai

from uuid import uuid4
from langsmith import traceable
from langsmith.run_trees import RunTree
from langsmith.wrappers import wrap_openai


In [16]:
messages = [
    { "role": "system", "content": "You are a helpful assistant. Please respond to the user's request only based on the given context." },
    { "role": "user", "content": "Is sunshine good for you?" }
]

In [17]:
# Collect run ID using RunTree
run_id = uuid4()
rt = RunTree(
    name="OpenAI Call RunTree",
    run_type="llm",
    inputs={"messages": messages},
    id=run_id
)
rt

RunTree(id=UUID('02ec5c39-45c6-4aad-8fbb-8fbc1c97c9a3'), name='OpenAI Call RunTree', start_time=datetime.datetime(2024, 3, 18, 11, 36, 46, 200445, tzinfo=datetime.timezone.utc), run_type='llm', end_time=None, extra={}, error=None, serialized={'name': 'OpenAI Call RunTree'}, events=None, inputs={'messages': [{'role': 'system', 'content': "You are a helpful assistant. Please respond to the user's request only based on the given context."}, {'role': 'user', 'content': 'Is sunshine good for you?'}]}, outputs=None, reference_example_id=None, parent_run_id=None, tags=None, parent_run=None, child_runs=[], session_name='default', session_id=None, client=Client (API URL: https://api.smith.langchain.com), dotted_order='20240318T113646200445Z02ec5c39-45c6-4aad-8fbb-8fbc1c97c9a3', trace_id=UUID('02ec5c39-45c6-4aad-8fbb-8fbc1c97c9a3'))

In [18]:
client = openai.Client()
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages
)
rt.end(outputs=chat_completion)
rt.post()
print("RunTree Run ID: ", run_id)

RunTree Run ID:  02ec5c39-45c6-4aad-8fbb-8fbc1c97c9a3


In [19]:
# Collect run ID using openai_wrapper
run_id = uuid4()
client = wrap_openai(openai.Client())
completion = client.chat.completions.create(
    model="gpt-3.5-turbo", messages=messages, langsmith_extra={
        "run_id": run_id,
    },
)
print("OpenAI Wrapper Run ID: ", run_id)

OpenAI Wrapper Run ID:  fbeb7d8b-d151-41d1-a177-9d322b556a04


In [20]:
# Collect run id using traceable decorator
run_id = uuid4()
@traceable(
    run_type="llm",
    name="OpenAI Call Decorator",
)
def call_openai(
    messages: list[dict], model: str = "gpt-3.5-turbo"
) -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages,
    ).choices[0].message.content
result = call_openai(
    messages,
    langsmith_extra={
        "run_id": run_id,
    },
)
print("Traceable Run ID: ", run_id)

Traceable Run ID:  1d93c6e2-c333-4347-820d-af845938c006


# 6.获取记录的 Run URL

运行将记录到您配置的任何项目（如果未设置，则为“默认”），您可以通过打开相应的项目详细信息页面来查看它们。

要以编程方式访问运行的 URL，您可以使用 LangSmith 客户端。

下面是一个示例。若要获取运行的运行 ID，可以按照此处的指南进行操作。

In [22]:
from langsmith import Client

client = Client()
run = client.read_run("1d93c6e2-c333-4347-820d-af845938c006")
print(run.url)

https://smith.langchain.com/o/1040bdf6-7131-5c03-b7be-fb1b4779027f/projects/p/5da73c3f-abb4-4b8e-8af4-51321f6b206a/r/1d93c6e2-c333-4347-820d-af845938c006?trace_id=1d93c6e2-c333-4347-820d-af845938c006&start_time=2024-03-18T11:38:48.849409


# 7.删除项目中的traces

In [ ]:
from langsmith import Client

client = Client()
client.delete_project(project_name="<project_name>")